In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LassoLars
from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.isotonic import IsotonicRegression
from sklearn.cross_decomposition import PLSRegression
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn import tree
import pandas as pd
import numpy as np
from whitebox.eval import WhiteBoxSensitivity
import time
import datetime

In [20]:
df = pd.read_csv("sample_classifier_data.csv")

In [21]:
ydepend = 'response'
df['column1.bin'] = df['column1'].apply(lambda x: 'bin_0' if x > 0 else 'bin_1')
df['column2.bin'] = df['column2'].apply(lambda x: 'bin_0' if x > 0 else 'bin_1')

string_categories = df.select_dtypes(include = ['O'])
# iterate over string categories
for cat in string_categories:
    df[cat] = pd.Categorical(df[cat])
    
df.select_dtypes(include = ['category'])
model_df = df.copy(deep = True)


dummies = pd.concat([pd.get_dummies(model_df.loc[:, col], prefix = col) for col in model_df.select_dtypes(include = ['category']).columns], axis = 1)
finaldf = pd.concat([model_df.select_dtypes(include = [np.number]), dummies], axis = 1)

In [22]:


#clf = RandomForestClassifier()
#clf = SVC()
#clf = BernoulliNB()
#clf = GaussianNB()
#clf = tree.DecisionTreeClassifier()
clf = PassiveAggressiveClassifier()
clf.fit(finaldf.loc[:, finaldf.columns != ydepend], df.loc[:, ydepend])


WB = WhiteBoxSensitivity(clf,
                    model_df=finaldf,
                    ydepend=ydepend,
                    cat_df=df,
                    featuredict=None,
                    groupbyvars=['column1.bin','column2.bin'],
                    aggregate_func=np.mean,
                    verbose=None,
                     std_num=2
                    )

    


c:\python27\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:84: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.passive_aggressive.PassiveAggressiveClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [23]:
starttime = time.time()
WB.run()
print "Run Time ",datetime.timedelta(seconds=time.time() - starttime)

Run Time  0:02:57.358000


In [24]:
WB.save('./test_lassolars_sensitivity.html')

In [25]:

from whitebox.eval import WhiteBoxError
WB = WhiteBoxError(clf,
                    model_df=finaldf,
                    ydepend=ydepend,
                    cat_df=df,
                    featuredict=None,
                    groupbyvars=['column1.bin','column2.bin'],
                    aggregate_func=np.mean,
                    verbose=None
                    )

In [26]:
starttime = time.time()
WB.run()
print "Run Time ",datetime.timedelta(seconds=time.time() - starttime)

Run Time  1:03:43.757000


In [27]:
WB.save('./test_passiveaggresive_error.html') 

In [ ]:
clf.fit(np.array(finaldf['column4']), np.array(finaldf[ydepend]))

In [ ]:
x